# Predicting the failure rate of Kickstarter projects

#### David Collins, Nathan Rosenberg

## Table of Contents <br>

_put table of contents here_

Rough outline:

Intro - David
 * background
 * packages and stuff

Data Wrangling - David

EDA
 * Plot graphs for 
 * which category of project is the most successful - David
     * take top 1 or 2 most successful categories of projects and graph which are the most successful within the sub-category
 * length of campaign vs failure rate
 * montary goal vs failure rate
 * see which factor has the highest correlation using multiple regression with both scikit-learn and statsmodels - David
 * build model to predict success or failure given these factors

Conclusion

# 1. Introduction

Something about what kickstarter is, about how theres a spread of projects failing or succeeding and we want to see what some potential causes for failed kickstarter projects may be, if there are any, using dataset taken from kaggle

Idk some basic bullshit

## Necessary Packages and Other Resources

Mention kaggle, links to documentation, used dataset 'ks-projects-201801.csv' because it contained more up to date data

Link to dataset https://www.kaggle.com/kemical/kickstarter-projects/data#ks-projects-201801.csv

In [41]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

Used latin-1 encoding because utf-8 can't encode certain characters (\x99 byte) in the CSV

In [42]:
# Read all of data from CSV file into a pandas dataframe
k_d = pd.read_csv('data.csv', encoding = 'latin-1')

k_d.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


Look at the different types of states

In [43]:
print(k_d['state'].unique())

['failed' 'canceled' 'successful' 'live' 'undefined' 'suspended']


Drop undefined and live since they can't necessarily be judged, consider failed, cancelled and suspended all failures

In [49]:
# Removing all the rows that have the state of undefined or live
k_d = k_d[k_d['state'] != 'undefined']
k_d = k_d[k_d['state'] != 'live']

# Add a column to the end of the dataframe and tag each row whether it was a success or failure 
# 1 = success, 0 = failure
k_d["successful"] = 0
k_d.loc[k_d['state'] == 'successful', 'successful'] = 1

k_d.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,successful
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95,0
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00,0
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00,0
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00,0
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00,0


# Data Exploration and Analysis

## Overall Failure Rates

In [65]:
failure_rate = k_d.groupby('successful')['successful'].count()
failure_rate

successful
0    238344
1    133956
Name: successful, dtype: int64

## Category Breakdown

In [66]:
category_totals = k_d.groupby('main_category')['main_category'].count()
total_entries = len(k_d)

